In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy import signal
import scipy.fftpack
from mpl_toolkits.mplot3d import Axes3D
import datetime
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline
import random
from sklearn.model_selection import cross_val_score 
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.ensemble import VotingClassifier,RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import pickle
from tabulate import tabulate

# `1. Importing`

In [2]:
walkSit = []
walkFall = []
runFall = []
downSit = []
freeFall = []
runSit = []
for i in range(3):
    walkSit.append(pd.read_csv("Data/testData/walkSit{}.csv".format(i+1), sep = ';'))

for i in range(3):    
    walkFall.append(pd.read_csv("Data/testData/walkFall{}.csv".format(i+1), sep = ';'))

for i in range(3):
    runFall.append(pd.read_csv("Data/testData/runFall{}.csv".format(i+1), sep = ';'))

for i in range(3):
    downSit.append(pd.read_csv("Data/testData/downSit{}.csv".format(i+1), sep = ';'))

for i in range(3):
    freeFall.append(pd.read_csv("Data/testData/freeFall{}.csv".format(i+1), sep = ';'))

# `2. Cleaning`

In [3]:
#using low pass filter 
b, a = signal.butter(3, 0.1, btype='lowpass', analog=False)

In [4]:
# walksit
low_passed_walkSit = []
for i in range(3):
    low_passed_walkSitX = list(signal.filtfilt(b, a, walkSit[i]["AccelerationX"]))
    low_passed_walkSitY = list(signal.filtfilt(b, a, walkSit[i]["AccelerationY"]))
    low_passed_walkSitZ = list(signal.filtfilt(b, a, walkSit[i]["AccelerationZ"]))
    low_passed_walkSit.append((pd.DataFrame(list(zip(low_passed_walkSitX, low_passed_walkSitY, low_passed_walkSitZ)),
                                            columns=["AccelerationX", "AccelerationY", "AccelerationZ"])))
for i in range(3):
    low_passed_walkSit[i]["AccelerationT"] = np.sqrt(walkSit[i]["AccelerationX"]**2 * 
                                                     walkSit[i]["AccelerationY"]**2 *
                                                     walkSit[i]["AccelerationZ"]**2 )

In [5]:
low_passed_walkFall = []
for i in range(3):
    low_passed_walkFallX = list(signal.filtfilt(b, a, walkFall[i]["AccelerationX"]))
    low_passed_walkFallY = list(signal.filtfilt(b, a, walkFall[i]["AccelerationY"]))
    low_passed_walkFallZ = list(signal.filtfilt(b, a, walkFall[i]["AccelerationZ"]))
    low_passed_walkFall.append((pd.DataFrame(list(zip(low_passed_walkFallX, low_passed_walkFallY, low_passed_walkFallZ)),
                                            columns=["AccelerationX", "AccelerationY", "AccelerationZ"])))

for i in range(3):
    low_passed_walkFall[i]["AccelerationT"] = np.sqrt(walkFall[i]["AccelerationX"]**2 * 
                                                      walkFall[i]["AccelerationY"]**2 *
                                                      walkFall[i]["AccelerationZ"]**2 )

In [6]:
low_passed_runFall = []
for i in range(3):
    low_passed_runFallX = list(signal.filtfilt(b, a, runFall[i]["AccelerationX"]))
    low_passed_runFallY = list(signal.filtfilt(b, a, runFall[i]["AccelerationY"]))
    low_passed_runFallZ = list(signal.filtfilt(b, a, runFall[i]["AccelerationZ"]))
    low_passed_runFall.append((pd.DataFrame(list(zip(low_passed_runFallX, low_passed_runFallY, low_passed_runFallZ)),
                                            columns=["AccelerationX", "AccelerationY", "AccelerationZ"])))
for i in range(3):
    low_passed_runFall[i]["AccelerationT"] = np.sqrt(runFall[i]["AccelerationX"]**2 * 
                                                      runFall[i]["AccelerationY"]**2 *
                                                      runFall[i]["AccelerationZ"]**2 )
    

In [7]:
low_passed_downSit = []
for i in range(3):
    low_passed_downSitX = list(signal.filtfilt(b, a, downSit[i]["AccelerationX"]))
    low_passed_downSitY = list(signal.filtfilt(b, a, downSit[i]["AccelerationY"]))
    low_passed_downSitZ = list(signal.filtfilt(b, a, downSit[i]["AccelerationZ"]))
    low_passed_downSit.append((pd.DataFrame(list(zip(low_passed_downSitX, low_passed_downSitY, low_passed_downSitZ)),
                                            columns=["AccelerationX", "AccelerationY", "AccelerationZ"])))
for i in range(3):
    low_passed_downSit[i]["AccelerationT"] = np.sqrt(downSit[i]["AccelerationX"]**2 * 
                                                      downSit[i]["AccelerationY"]**2 *
                                                      downSit[i]["AccelerationZ"]**2 )

In [8]:
low_passed_freeFall = []
for i in range(3):
    low_passed_freeFallX = list(signal.filtfilt(b, a, freeFall[i]["AccelerationX"]))
    low_passed_freeFallY = list(signal.filtfilt(b, a, freeFall[i]["AccelerationY"]))
    low_passed_freeFallZ = list(signal.filtfilt(b, a, freeFall[i]["AccelerationZ"]))
    low_passed_freeFall.append((pd.DataFrame(list(zip(low_passed_freeFallX, low_passed_freeFallY, low_passed_freeFallZ)),
                                            columns=["AccelerationX", "AccelerationY", "AccelerationZ"])))
for i in range(3):
    low_passed_freeFall[i]["AccelerationT"] = np.sqrt(freeFall[i]["AccelerationX"]**2 * 
                                                      freeFall[i]["AccelerationY"]**2 * freeFall[i]["AccelerationZ"]**2)

# `3. Transformation`

In [9]:
def stats_generator(df):
    # total max Acceleration
    max_AT = df['AccelerationT'].idxmax()
    
    # 1 sec before and after drop data
    Imp_Data = df[max_AT - 50:max_AT + 25] 
    Imp_Data = pd.DataFrame(Imp_Data)
    
    MeanX = Imp_Data['AccelerationX'].mean()
    MeanY = Imp_Data['AccelerationY'].mean()
    MeanZ = Imp_Data['AccelerationZ'].mean()
    MeanT = Imp_Data['AccelerationT'].mean()
    
    StdX = Imp_Data['AccelerationX'].std()
    StdY = Imp_Data['AccelerationY'].std()
    StdZ = Imp_Data['AccelerationZ'].std()
    StdT = Imp_Data['AccelerationT'].std()
    
    MinX = Imp_Data['AccelerationX'].min()
    MinY = Imp_Data['AccelerationY'].min()
    MinZ = Imp_Data['AccelerationZ'].min()
    MinT = Imp_Data['AccelerationT'].min()
    
    MaxX = Imp_Data['AccelerationX'].max()
    MaxY = Imp_Data['AccelerationY'].max()
    MaxZ = Imp_Data['AccelerationZ'].max()
    MaxT = Imp_Data['AccelerationT'].max()
    
    stat = [{'MeanX':MeanX, 'MeanY':MeanY, 'MeanZ':MeanZ, 'MeanT':MeanT, 
             'StdX': StdX, 'StdY': StdY, 'StdZ': StdZ, 'StdT': StdT,
             'MinX': MinX, 'MinY': MinY, 'MinZ': MinZ, 'MinT': MinT,
             'MaxX':MaxX, 'MaxY':MaxY, 'MaxZ':MaxZ, 'MaxT':MaxT}]
    
    return pd.DataFrame(stat)

In [10]:
stat_walkSit = stats_generator(low_passed_walkSit[0])
for i in range(2):
    stat_walkSit = stat_walkSit.append(stats_generator(low_passed_walkSit[i + 1]))
stat_walkSit = stat_walkSit.dropna()
stat_walkSit = stat_walkSit.reset_index(drop=True)
stat_walkSit ['Fall'] = 0
# fall 0 mean false i.e. no fall

In [11]:
stat_walkFall = stats_generator(low_passed_walkFall[0])
for i in range(2):
    stat_walkFall = stat_walkFall.append(stats_generator(low_passed_walkFall[i + 1]))
stat_walkFall = stat_walkFall.dropna()
stat_walkFall = stat_walkFall.reset_index(drop=True)
stat_walkFall ['Fall'] = 1
# fall 0 mean false i.e. no fall

In [12]:
stat_runFall = stats_generator(low_passed_runFall[0])
for i in range(2):
    stat_runFall = stat_runFall.append(stats_generator(low_passed_runFall[i + 1]))
stat_runFall = stat_walkFall.dropna()
stat_runFall = stat_walkFall.reset_index(drop=True)
stat_runFall ['Fall'] = 1
# fall 0 mean false i.e. no fall

In [13]:
stat_downSit = stats_generator(low_passed_downSit[0])
for i in range(2):
    stat_downSit = stat_downSit.append(stats_generator(low_passed_downSit[i + 1]))
stat_downSit = stat_downSit.dropna()
stat_downSit = stat_downSit.reset_index(drop=True)
stat_downSit ['Fall'] = 0
stat_downSit
# fall 0 mean false i.e. no fall

,MeanX,MeanY,MeanZ,MeanT,StdX,StdY,StdZ,StdT,MinX,MinY,MinZ,MinT,MaxX,MaxY,MaxZ,MaxT,Fall
0,-0.542930,0.904579,-0.303651,0.467309,0.343317,0.481893,0.448516,0.896633,-1.301513,-0.419411,-1.211874,0.000421,0.048762,1.726148,0.097651,4.896431,0
1,-0.531983,1.569879,-0.046293,2.520955,0.461949,0.398333,0.155055,7.833440,-1.230656,0.859806,-0.415732,0.000082,0.264583,2.371279,0.158859,48.845084,0
2,-0.579644,1.635359,-0.035430,1.029004,0.391223,0.407906,0.129970,2.034076,-1.208973,0.865597,-0.328769,0.001573,0.180888,2.343828,0.157854,10.453938,0


In [14]:
stat_freeFall = stats_generator(low_passed_freeFall[0])
for i in range(2):
    stat_freeFall = stat_freeFall.append(stats_generator(low_passed_freeFall[i + 1]))
stat_freeFall = stat_freeFall.dropna()
stat_freeFall = stat_freeFall.reset_index(drop=True)
stat_freeFall ['Fall'] = 1
# fall 1 mean false i.e. fall

# `4. Importing the trained models`

In [15]:
randomForestClf = pickle.load(open("Models/RandomForestClf.pkl", "rb"))

In [16]:
adaBoostclf = pickle.load(open("Models/AdaBoostClf.pkl", "rb"))

In [17]:
MlpClf = pickle.load(open("Models/MlpClf.pkl", "rb"))

# 5. `Predicting`

In [18]:
# 1. walkfall 
X = stat_walkFall.drop("Fall", axis = 1)
rfResults = randomForestClf.predict(X)
adaResults = adaBoostclf.predict(X)
mlpResults = MlpClf.predict(X)
wfResults = pd.DataFrame()
wfResults['Random forest'] = rfResults
wfResults['AdaBoost'] = adaResults
wfResults['MLP'] = mlpResults
print("Walk and fall predictions: ")
print(tabulate(wfResults, headers='keys', tablefmt="pretty", showindex=False))

Walk and fall predictions: 
+---------------+----------+-----+
| Random forest | AdaBoost | MLP |
+---------------+----------+-----+
|       1       |    1     |  0  |
|       1       |    1     |  1  |
|       1       |    1     |  1  |
+---------------+----------+-----+


In [19]:
# 2. walksit 
X = stat_walkSit.drop("Fall", axis = 1)
rfResults = randomForestClf.predict(X)
adaResults = adaBoostclf.predict(X)
mlpResults = MlpClf.predict(X)
wsResults = pd.DataFrame()
wsResults['Random forest'] = rfResults
wsResults['AdaBoost'] = adaResults
wsResults['MLP'] = mlpResults
print("Walk and sit predictions: ")
print(tabulate(wsResults, headers='keys', tablefmt="pretty", showindex=False))

Walk and sit predictions: 
+---------------+----------+-----+
| Random forest | AdaBoost | MLP |
+---------------+----------+-----+
|       0       |    0     |  0  |
|       0       |    0     |  0  |
|       0       |    0     |  0  |
+---------------+----------+-----+


In [20]:
# 4. down Sit 
X = stat_downSit.drop("Fall", axis = 1)
rfResults = randomForestClf.predict(X)
adaResults = adaBoostclf.predict(X)
mlpResults = MlpClf.predict(X)
wsResults = pd.DataFrame()
wsResults['Random forest'] = rfResults
wsResults['AdaBoost'] = adaResults
wsResults['MLP'] = mlpResults
print("Downstairs and sit predictions: ")
print(tabulate(wsResults, headers='keys', tablefmt="pretty", showindex=False))

Downstairs and sit predictions: 
+---------------+----------+-----+
| Random forest | AdaBoost | MLP |
+---------------+----------+-----+
|       1       |    1     |  0  |
|       1       |    1     |  0  |
|       1       |    1     |  0  |
+---------------+----------+-----+


In [21]:
# 4. stat_runFall 
X = stat_runFall.drop("Fall", axis = 1)
rfResults = randomForestClf.predict(X)
adaResults = adaBoostclf.predict(X)
mlpResults = MlpClf.predict(X)
wsResults = pd.DataFrame()
wsResults['Random forest'] = rfResults
wsResults['AdaBoost'] = adaResults
wsResults['MLP'] = mlpResults
print("Run and fall predictions: ")
print(tabulate(wsResults, headers='keys', tablefmt="pretty", showindex=False))

Run and fall predictions: 
+---------------+----------+-----+
| Random forest | AdaBoost | MLP |
+---------------+----------+-----+
|       1       |    1     |  0  |
|       1       |    1     |  1  |
|       1       |    1     |  1  |
+---------------+----------+-----+


In [22]:
# 5. walksit 
X = stat_freeFall.drop("Fall", axis = 1)
rfResults = randomForestClf.predict(X)
adaResults = adaBoostclf.predict(X)
mlpResults = MlpClf.predict(X)
wsResults = pd.DataFrame()
wsResults['Random forest'] = rfResults
wsResults['AdaBoost'] = adaResults
wsResults['MLP'] = mlpResults
print("Freefall predictions: ")
print(tabulate(wsResults, headers='keys', tablefmt="pretty", showindex=False))

Freefall predictions: 
+---------------+----------+-----+
| Random forest | AdaBoost | MLP |
+---------------+----------+-----+
|       1       |    1     |  1  |
|       1       |    1     |  1  |
|       1       |    1     |  1  |
+---------------+----------+-----+


In [23]:
# exploring downsit

In [24]:
# 4. down Sit 
X = stat_downSit.drop("Fall", axis = 1)
rfResults = randomForestClf.predict_proba(X)
adaResults = adaBoostclf.predict_proba(X)
mlpResults = MlpClf.predict_proba(X)

pd.set_option('precision', 0)
pd.set_option('display.float_format', lambda x: '%.0f' % x)


rfResultsProba = pd.DataFrame()
rfResultsProba["Not-Fall"] = rfResults[:,0]
rfResultsProba["Fall"] = rfResults[:,1]
print("Pridicted probability by Random forest : ")
print(tabulate(rfResultsProba, headers='keys', tablefmt="pretty", showindex=False))

rfResultsProba = pd.DataFrame()
rfResultsProba["Not-Fall"] = adaResults[:,0]
rfResultsProba["Fall"] = adaResults[:,1]
print("Pridicted probability by Ada Boost : ")
print(tabulate(rfResultsProba, headers='keys', tablefmt="pretty", showindex=False))

rfResultsProba = pd.DataFrame()

rfResultsProba["Not-Fall"] = mlpResults[:,0]
rfResultsProba["Fall"] = mlpResults[:,1]
print("Pridicted probability by MLP : ")
print(tabulate(rfResultsProba, headers='keys', tablefmt="pretty", showindex=False))

Pridicted probability by Random forest : 
+----------+------+
| Not-Fall | Fall |
+----------+------+
|   0.17   | 0.83 |
|   0.27   | 0.73 |
|   0.33   | 0.67 |
+----------+------+
Pridicted probability by Ada Boost : 
+----------------------+--------------------+
|       Not-Fall       |        Fall        |
+----------------------+--------------------+
| 0.055932664219750385 | 0.9440673357802496 |
| 0.32183885942881235  | 0.6781611405711877 |
| 0.32183885942881235  | 0.6781611405711877 |
+----------------------+--------------------+
Pridicted probability by MLP : 
+--------------------+------------------------+
|      Not-Fall      |          Fall          |
+--------------------+------------------------+
| 0.9999986863898758 | 1.3136101242371043e-06 |
| 0.999017144449047  |  0.00098285555095297   |
| 0.9999986549742913 |  1.34502570874002e-06  |
+--------------------+------------------------+
